In [1]:
import random
import numpy as np
import copy 
import queue
import matplotlib.pyplot as plt
import pulp
import pandas as pd
import time 

class Building:
    counter = 0
    def __init__(self, num_resources=3, max_build_count=(1, 15), building_size_bounds=(1, 50), infinite_prob=0.0, infinite_buildings=False, struct_value_bounds=(1, 5)):
        self.name = f"{Building.counter}"
        Building.counter += 1
        self.resource_values = np.random.randint(*struct_value_bounds, size=num_resources)
        if infinite_buildings or np.random.rand() < infinite_prob:
            self.max_build_count = 99999
        else:
            self.max_build_count = np.random.randint(*max_build_count)
        self.building_size = np.random.randint(*building_size_bounds)
        self.built_locations = []
        self.build_rem = self.max_build_count

    def display(self):
        print()
        print(f"Name: {self.name}")
        print(f"Building Resource Values: {self.resource_values}")
        print(f"Max Build Count: {self.max_build_count}")
        print(f"Building Size: {self.building_size}")
        print(f"Built Locations: {self.built_locations}")
        print(f"Building_Remaining: {self.build_rem}")
        print()
    

class Land:
    counter = 0
    def __init__(self, space_range=(100, 500), num_resources=3, mean=.5, std=.2, bounds=(0,2), rounding=2):
        self.name = f"L{Land.counter}"
        Land.counter += 1
        self.space = np.random.randint(space_range[0], space_range[1] + 1)
        # Make guassian distribution between 0 and 2 centered around .5 with standard dev of .2 (if using default values)
        self.resource_values = np.round(np.clip(np.random.normal(mean, std, num_resources), bounds[0], bounds[1]), rounding)
        self.rem_space = self.space
        self.built_buildings = []
        self.building_prod = [0]*num_resources
        self.calc_prod = [0]*num_resources

    def construct_building(self, building):
        # Run a check to see if the building can be made when looking at size/build contraints
        if building.build_rem > 0 and building.building_size <= self.rem_space:
            # Update Land variables
            self.rem_space -= building.building_size
            self.built_buildings.append(building.name)

            # Update Building variables
            building.build_rem -= 1
            building.built_locations.append(self.name)

            # Update resource production
            self.building_prod += building.resource_values
            self.calc_prod = self.resource_values * self.building_prod
            
    def remove_building(self, building):
        if building.name in self.built_buildings:
            # Update Land variables
            self.rem_space += building.building_size
            self.built_buildings.remove(building.name)

            # Update Building variables
            building.build_rem += 1
            building.built_locations.remove(self.name)

            # Update resource production
            self.building_prod -= building.resource_values
            self.calc_prod = self.resource_values * self.building_prod


    def display(self):
        print()
        print(f"Name: {self.name}")
        print(f"Space: {self.space}")
        print(f"Resource Values: {self.resource_values}")
        print(f"Remaining_Space: {self.rem_space}")
        print(f"Built Buildings List: {self.built_buildings}")
        print(f"Building Production: {self.building_prod}")
        print(f"Calc Production: {self.calc_prod}")
        print()

In [2]:
# Random Algorithm to distribute buildings randomly among lands
# Algorithm Summary: Choose a piece of Land -> Fill Land with random buildings -> When full move to next land and repeat
def random_algorithm(buildings, lands):
    lands_copy = copy.deepcopy(lands)
    buildings_copy = copy.deepcopy(buildings)
    for land in lands_copy:
        while land.rem_space > 0:
            # Filter available buildings that can fit in the remaining space
            available_buildings = [b for b in buildings_copy if b.building_size <= land.rem_space and b.build_rem > 0]
            # No available buildings that can fit
            if not available_buildings:
                break
            building = random.choice(available_buildings)
            land.construct_building(building)
    return buildings_copy, lands_copy

# Greedy Algorithm to greedyly choose best building per land
# Algorithm Summary: Create queue of best land based on resource values, then create queue based on best building/space efficiency,
# place buildings until queue empty or land is filled repeat for all land
def greedy_algorithm(buildings, lands):
    lands_copy = copy.deepcopy(lands)
    buildings_copy = copy.deepcopy(buildings)

    # Sort lands by total resource values, richest land is at the top of the queue
    lands_copy.sort(key=lambda land: sum(land.resource_values), reverse=True)

    for land in lands_copy:
        # Sort buildings by space efficiency (resource values*land resources)/building size
        buildings_copy.sort(key=lambda building: sum((land.resource_values * building.resource_values)/building.building_size), reverse=True)

        # Add most efficent building until no longer possible
        for building in buildings_copy:
            while (land.rem_space >= building.building_size) and building.build_rem >= 1:
                land.construct_building(building)

    return buildings_copy, lands_copy

def lp_algorithm(buildings, lands):
    model = pulp.LpProblem("Building_Allocation", pulp.LpMaximize)
    # Create a dictionary to hold decision variables
    building_vars = {}
    for building in buildings:
        for land in lands:
            building_vars[(building.name, land.name)] = pulp.LpVariable(
        f"B_{building.name}_on_L{land.name}", lowBound=0, upBound=building.max_build_count, cat='Integer'
    )

    objective = []
    for land in lands:
        land_richness = land.resource_values
        for building in buildings:
            building_production = building.resource_values
            for i in range(len(land_richness)):
                objective.append(
                    building_vars[(building.name, land.name)] * land_richness[i] * building_production[i]
                )

    model += pulp.lpSum(objective), "Objective"
    for land in lands:
        land_space = land.space
        space_constraint = pulp.lpSum(
            building_vars[(building.name, land.name)] * building.building_size for building in buildings
        )
        model += space_constraint <= land_space, f"Land_{land.name}_Space_Constraint"
        
    for building in buildings:
        build_count = building.build_rem
        usage_constraint = pulp.lpSum(building_vars[(building.name, land.name)] for land in lands)
        model += usage_constraint <= build_count, f"Building_{building.name}_Usage_Constraint"

    model.solve(pulp.PULP_CBC_CMD(msg=False, timeLimit=30))
    
    #if pulp.LpStatus[model.status] == "Optimal":
    #    for (building_name, land_name), var in building_vars.items():
    #        if var.varValue > 0:
    #            print(f"Allocate {var.varValue} times Building {building_name} on Land {land_name}")
    
    return building_vars

def calculate_LP_production(buildings, lands, building_vars):
    num_resources = len(lands[0].resource_values)
    production_scores = [0.0] * num_resources

    for land in lands:
        land_richness = land.resource_values
        for building in buildings:
            building_production = building.resource_values
            for i in range(num_resources):
                production_scores[i] += (
                    building_vars[(building.name, land.name)].varValue
                    * land_richness[i]
                    * building_production[i]
                )

    return [round(score, 2) for score in production_scores]

def is_feasible_swap(land1, land2, building1, building2):
    return (land1.rem_space + building1.building_size >= building2.building_size and
            land2.rem_space + building2.building_size >= building1.building_size and
            building1.build_rem > 0 and building2.build_rem > 0)

def hill_climbing(buildings, lands, max_iterations=100000):
    best_buildings = copy.deepcopy(buildings)
    best_lands = copy.deepcopy(lands)
    best_score = calculate_total_production(lands, num_resources)
    scores = []
    for i in range(max_iterations):
        # Randomly select two buildings
        building1 = random.choice(best_buildings)
        building2 = random.choice(best_buildings)

        # Randomly select two lands
        land1 = random.choice(best_lands)
        land2 = random.choice(best_lands)

        # See if they can be swapped
        if is_feasible_swap(land1, land2, building1, building2):
            land1.remove_building(building1)
            land1.construct_building(building2)

            land2.remove_building(building2)
            land2.construct_building(building1)

            new_score = calculate_total_production(best_lands, num_resources)

            # If the new score is better, keep the swap
            if sum(new_score) > sum(best_score):
                best_score = new_score
            else:
                # Revert the swap if the new score is not better
                land1.remove_building(building2)
                land1.construct_building(building1)
                land2.remove_building(building1)
                land2.construct_building(building2)
    return best_score

def calculate_total_production(lands, num_resources):
    total_production = np.zeros(num_resources)
    for land in lands:
        total_production += land.calc_prod
    return total_production

def sparsity_score(max_build_count, building_size_bounds, space_range, num_buildings, num_land):
    avg_land_space_range = (space_range[0] + space_range[1]) / 2.0
    avg_building_size = (building_size_bounds[0] + building_size_bounds[1]) / 2.0
    avg_building_count = (max_build_count[0] + max_build_count[1]) / 2.0
    sparsity_ratio = (avg_building_count * avg_building_size * num_buildings) / (avg_land_space_range * num_land)
    return sparsity_ratio

In [3]:
############################### Problem Space Variables ################################

# Prototype params to forumlate problem space, there are the default params

# Resources Params
num_resources=5

# Building Params
max_build_count=(5, 20)
building_size_bounds=(1, 5)
infinite_prob=0.0
infinite_buildings=False
struct_value_bounds=(1, 5)

# Land Params
space_range = (100, 200)
mean=.5
std=.2
bounds=[0,2]
rounding=2

# Problem size 
# Where sparsity score is a value between (0,inf) which detriminates some ratio between total building space available/total land space
num_buildings = 20
num_land = 5
# If True Provide weighting scheme of resources ex [1, 0, 0] means you are only optimizing for resource 0, 
optimize_resource = False
resource_weights = [1]*num_resources

In [4]:
small = [5, 2, 2, "Small"]
normal_ = [8, 4, 3, "Normal"]
large = [20, 7, 4, "Large"]

v_sparse = [(1, 5), (5, 20)]
sparse = [(3, 8), (5, 20)]
normal = [(5, 15), (5, 20)]
rich = [(10, 20), (5, 20)]
v_rich = [(1, 5), (9999, 99999)]

seeds = [i for i in range(10)]

score = sparsity_score(max_build_count, building_size_bounds, space_range, num_buildings, num_land)
print(f"Sparsity Score: {score}")

Sparsity Score: 1.0


In [5]:
scores_df = pd.DataFrame(columns=["Greedy_Score", "Random_Score", "Lp_Score", "Hill_Score", "Sparsity_Score", "Problem_Space"])
runtime_df = pd.DataFrame(columns=["Greedy_Time", "Random_Time", "Lp_Time", "Hill_Time", "Sparsity_Score", "Problem_Space"])

In [6]:
for space in [small, normal_, large]:
    for b in [v_sparse, sparse, normal, rich, v_rich]:
        greedy_score = 0
        random_score = 0
        lp_score = 0
        hill_score = 0

        greedy_time = 0
        random_time = 0
        lp_time = 0
        hill_time = 0

        for s in seeds:
            print(s)
            num_buildings = space[0]
            num_land = space[1]
            num_resources = space[2]
            building_size_bounds = b[0]
            max_build_count = b[1]
            #print("===================================")
            #score = sparsity_score(max_build_count, building_size_bounds, space_range, num_buildings, num_land)
            #print(f"Sparsity Score: {score}")
            random_seed = s
            random.seed(random_seed)
            np.random.seed(random_seed)
            buildings = [Building(max_build_count=max_build_count, num_resources=num_resources, building_size_bounds=building_size_bounds, infinite_prob=infinite_prob, infinite_buildings=infinite_buildings, struct_value_bounds=struct_value_bounds) for _ in range(num_buildings)]
            lands = [Land(space_range=space_range, num_resources=num_resources, mean=mean, std=std, bounds=bounds, rounding=rounding) for _ in range(num_land)]\

            # Calculate and display total production score
            
            start_time = time.time()
            random_buildings, random_land = random_algorithm(buildings, lands)
            total_production_score = calculate_total_production(random_land, num_resources)
            #print(f"Total Production Score with Random Algorithm: {total_production_score} \nTotal Production: {round(sum(total_production_score), 2)}")
            random_score += sum(total_production_score)
            end_time = time.time()
            random_time += end_time - start_time

            start_time = time.time()
            greedy_buildings, greedy_land = greedy_algorithm(buildings, lands)
            total_production_score = calculate_total_production(greedy_land, num_resources)
            greedy_score += sum(total_production_score)
            #print(f"Total Production Score with Greedy Algorithm: {total_production_score} \nTotal Production: {round(sum(total_production_score), 2)}")
            end_time = time.time()
            greedy_time += end_time - start_time

            start_time = time.time()
            building_vars = lp_algorithm(buildings, lands)
            total_production_score = calculate_LP_production(buildings, lands, building_vars)
            #print(f"Total Production Score with LP Algorithm: {total_production_score} \nTotal Production: {round(sum(total_production_score), 2)}")
            lp_score += sum(total_production_score)
            end_time = time.time()
            lp_time += end_time - start_time

            start_time = time.time()
            best_score = hill_climbing(buildings, lands)
            #print(f"Total Production Score with Hill Climbing Algorithm: {best_score} \nTotal Production: {round(sum(best_score), 2)}")
            hill_score += sum(best_score)
            end_time = time.time()
            hill_time += end_time - start_time

        score = sparsity_score(max_build_count, building_size_bounds, space_range, num_buildings, num_land)
        if score > 10:
            score = "inf"
        else:
            score = round(score,2)
        scores_df.loc[len(scores_df)] = [round(greedy_score/lp_score, 6) * 10, round(random_score/lp_score, 6) * 10, round(lp_score/lp_score, 6) * 10, round(hill_score/lp_score, 6) * 10, score, space[3]]
        runtime_df.loc[len(scores_df)] = [greedy_time, random_time, lp_time, hill_time, score, space[3]]

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


In [7]:
scores_df

,Greedy_Score,Random_Score,Lp_Score,Hill_Score,Sparsity_Score,Problem_Space
0,9.87580,8.28383,10.0,8.53884,0.62,Small
1,9.77337,9.12159,10.0,8.74857,1.15,Small
2,9.73566,7.69735,10.0,7.24776,2.08,Small
3,9.82492,7.37753,10.0,7.77179,3.12,Small
4,10.00000,4.29022,10.0,4.18662,inf,Small
5,9.83704,8.15879,10.0,8.16262,0.5,Normal
6,9.73373,9.05372,10.0,8.97992,0.92,Normal
7,9.73762,7.92208,10.0,8.18155,1.67,Normal
8,9.63522,7.30792,10.0,7.90261,2.5,Normal
9,10.00000,3.57556,10.0,3.59158,inf,Normal


In [8]:
runtime_df

,Greedy_Time,Random_Time,Lp_Time,Hill_Time,Sparsity_Score,Problem_Space
1,0.005002,0.001997,0.205996,2.639678,0.62,Small
2,0.005996,0.001999,0.222017,3.963342,1.15,Small
3,0.006001,0.002002,0.232238,9.889914,2.08,Small
4,0.002999,0.001000,0.220365,12.684748,3.12,Small
5,0.005997,0.009000,0.193007,14.297735,inf,Small
6,0.004000,0.005996,0.211007,1.591262,0.5,Normal
7,0.002999,0.007001,0.220009,1.759223,0.92,Normal
8,0.004002,0.004995,0.524003,6.797316,1.67,Normal
9,0.004001,0.004011,0.732998,10.322866,2.5,Normal
10,0.011000,0.009998,0.216006,13.782939,inf,Normal


In [9]:
runtime_df.to_csv(path_or_buf="runtime_ez.csv", index = False)
scores_df.to_csv(path_or_buf="scores_ez.csv", index = False)